Install and import **packages**

In [1]:
!pip install datasets
!pip install transformers
!pip install accelerate -U

In [2]:
from datasets import load_dataset, load_metric
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import TrainingArguments, Trainer
from transformers import pipeline
import json
import os
from google.colab import drive

Mount your google drive

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Create model path

In [4]:
models_path = '/content/gdrive/My Drive/models/'
model_name = 'sentiment-analysis'
model_path = os.path.join(models_path,model_name)
print(model_path)

/content/gdrive/My Drive/models/sentiment-analysis


In [11]:
raw_dataset = load_dataset('glue', 'sst2')

  0%|          | 0/3 [00:00<?, ?it/s]

In [12]:
checkpoint = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [13]:
def tokenize_fn(batch):
    return tokenizer(batch['sentence'], truncation=True)

In [14]:
tokenized_dataset = raw_dataset.map(tokenize_fn, batched=True)

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

In [15]:
training_args = TrainingArguments(
    'my_trainer',
    evaluation_strategy='epoch',
    num_train_epochs=1,
    save_strategy='epoch'
)

In [16]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
params_before = []
for param in model.parameters():
    params_before.append(param.clone())

In [17]:
metric = load_metric('glue', 'sst2')
metric.compute(predictions=[1,0,1],references=[1,0,0])

<ipython-input-17-edca1687a6d6>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('glue', 'sst2')


{'accuracy': 0.6666666666666666}

In [18]:
def compute_metrics(logits_and_labels):
    logits, labels = logits_and_labels
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [19]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [20]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.215500,0.353244,0.907110


TrainOutput(global_step=8419, training_loss=0.26421211207333534, metrics={'train_runtime': 416.316, 'train_samples_per_second': 161.774, 'train_steps_per_second': 20.223, 'total_flos': 518596929468840.0, 'train_loss': 0.26421211207333534, 'epoch': 1.0})

In [23]:
trainer.save_model(model_path)

In [24]:
newmodel = pipeline('text-classification', model=model_path)


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [25]:
newmodel('This movie is awful!')

[{'label': 'LABEL_0', 'score': 0.9992818236351013}]

In [26]:
#fix config file
#config_path = 'content/my_trainer/config.json'
config_path = os.path.join(model_path,'config.json')
with open(config_path, 'r') as f:
    config = json.load(f)

In [27]:
config['id2label'] = {0: 'NEGATIVE', 1: 'POSITIVE'}


In [28]:
with open(config_path, 'w') as f:
    json.dump(config, f)

In [5]:
newmodel = pipeline('text-classification', model=model_path, device=0)


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [6]:
newmodel('This movie is great')

[{'label': 'POSITIVE', 'score': 0.9990754127502441}]